In [96]:
!pip install pillow-heif
import torch
import requests
import numpy as np
from google.colab import drive
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd

#for authing drive, and file hadnling on drive
import gspread
from google.auth import default
from google.colab import auth

from googleapiclient.http import MediaIoBaseDownload
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#for opening images, and heic files
from PIL import Image
import pillow_heif
from pillow_heif import register_heif_opener

import sqlite3
from sqlite3 import Error
from contextlib import closing
from google.colab import userdata
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
# #code for creating sqlite database in our shared folder. Only need to run this once

db_path = userdata.get('staves_db_path')
# with closing (sqlite3.connect(db_path)) as conn:
#   with closing(conn.cursor()) as cursor:
#     cursor.execute("""
#     CREATE TABLE downloadedStavesData (
#       photo_name text NOT NULL,
#       stave_count integer NOT NULL
#     )
#     """)

In [98]:
#pipeline code
#TODO: test it in local to see if it works with pathing and auth
class Gsheet_data_download:
  #paths should be paths from google colab to google drive
  def __init__(self, gSheet_path, download_dir_path, db_path, db_table_name="downloadedStavesData"):
    self.gSheet_path = gSheet_path
    self.download_dir_path = download_dir_path
    os.makedirs(download_dir_path, exist_ok=True)
    drive.mount('/content/drive')

    #auth_obj for reading into google sheet, pydrive obj for downladoing with img links
    self.auth_obj, self.pydrive_obj = self.auth_user()
    self.sheet_obj = self.connect_to_gsheet()
    self.db_path = db_path
    self.db_table_name = db_table_name
    self.connected_to_gsheet = False

    #get downloaded img name from sqlite table so we don't download them again
    self.downloaded_img_names = self.read_sqlite_table()['photo_name'].values.tolist()
    print(self.downloaded_img_names)

  #don't use this publicly haha
  def auth_user(self):
    try:
      auth.authenticate_user()
      #use this to get current user's credentials, for google colab
      creds, _ = default()
      #create auth obj for g sheet python api for google colab
      gc = gspread.authorize(creds)

      #need to also inst pydrive wrapper obj for accessing img links:
      gauth = GoogleAuth()
      gauth.credentials = GoogleCredentials.get_application_default()
      drive_obj = GoogleDrive(gauth)

      return gc, drive_obj
    except Exception as e:
      print(f"Error from auth_user of Gsheet_data_download_split: {e}")
      return False

  def connect_to_gsheet(self):
    try:
      connected_sheet = self.auth_obj.open_by_url(self.gSheet_path)
      self.connected_to_gsheet = True
      return connected_sheet
    except Exception as e:
      print(f"Error from connect_to_gsheet of Gsheet_data_download_split: {e}")
      return False

  ###+++++
  # user callable functions
  ###+++++
  def read_sheet_into_pd(self):
    try:
      worksheet = self.sheet_obj.get_worksheet(0)

      # get_all_values gives a list of rows.
      rows = worksheet.get_all_values()
      print(rows)
      sheet_df = pd.DataFrame.from_records(rows)
      return sheet_df
    except Exception as e:
      print(f"Error from read_sheet_into_pd of Gsheet_data_download_split: {e}")
      return False

  def get_imgs_and_split(self):
    try:
      sheet_df = self.read_sheet_into_pd()
      train_df, test_df = train_test_split(sheet_df, test_size=0.2)
      return train_df, test_df
    except Exception as e:
      print(f"Error from get_imgs_and_split of Gsheet_data_download_split: {e}")
      return False

  #helper func for downloading one df worth of imgs
  def download_imgs_from_one_df(self, input_df, download_path):
    try:
      with closing(sqlite3.connect(self.db_path)) as conn:
        with closing(conn.cursor()) as cursor:
          count = 0
          for index, row in input_df.iterrows():
              file_id = row[1]
              file_id = file_id.split('id=')[-1] #must do this since id is embedded in the url
              stave_count = row[2]
              try:
                #need to use pydrive object to create the file object first, which is then used for downloading to local
                file_obj = self.pydrive_obj.CreateFile({'id': file_id})

                file_metaData = file_obj.FetchMetadata(fetch_all=True)
                file_name, file_ext = os.path.splitext(file_obj['title'])

                #heic conversion: download as heic, use pillow-heif to convert to jpg
                if file_ext.lower() == ".heic":
                  print(f"{file_name}")
                  heic_file_path = os.path.join(download_path, f"{file_name}.heic")
                  file_obj.GetContentFile(heic_file_path)
                  heif_file = pillow_heif.read_heif(heic_file_path)
                  image = Image.frombytes(heif_file.mode, heif_file.size,
                                        heif_file.data, "raw", heif_file.mode, heif_file.stride,)
                  jpeg_file_path = os.path.join(download_path, f"{file_name}HEIC.jpg")
                  image.save(jpeg_file_path, "JPEG")
                  print(f"Successfully converted {heic_file_path} to {jpeg_file_path}")
                  os.remove(heic_file_path)

                else:
                  # #GetContentFile() for downloading from drive to local
                  file_obj.GetContentFile(os.path.join(download_path, f"{file_name}.jpg"))

                with open(os.path.join(download_path, f"staves_count.txt"), 'a') as f:
                  f.write(f"{file_name}:{str(stave_count)}")
                  f.write("\n")

                print(f"Successfully downloaded {file_id}.jpg to {download_path}")

                #update the table now
                self.update_table_with_current_img_download(file_name, stave_count)

                # count += 1
                # if count > 30:
                #   return
              except Exception as e:
                print(f"Failed to download file {file_id}: {e}")
    except Exception as e:
      print(f"Error from download_imgs_from_one_df of Gsheet_data_download_split: {e}")
      return False

  def download_imgs_without_split(self):
    try:
      sheet_df = self.read_sheet_into_pd()
      self.download_imgs_from_one_df(sheet_df, self.download_dir_path)
      return True
    except Exception as e:
      print(f"Error from download_imgs_without_split of Gsheet_data_download_split: {e}")
      return False

  def download_imgs_from_split_dfs(self):
    try:
      train_df, test_df = self.get_imgs_and_split()
      self.download_imgs_from_one_df(train_df, os.path.join(self.download_dir_path, "train"))
      self.download_imgs_from_one_df(test_df, os.path.join(self.download_dir_path, "test"))
      return True
    except Exception as e:
      print(f"Error from download_imgs_from_split_dfs of Gsheet_data_download_split: {e}")
      return False

  ###SQLITE section###
  def read_sqlite_table(self):
    try:
      with closing(sqlite3.connect(self.db_path)) as conn:
        with closing(conn.cursor()) as cursor:
          select_qry = f"""
          SELECT * FROM {self.db_table_name}
          """
          result_pd = pd.read_sql_query(select_qry, conn)
          return result_pd
    except Exception as e:
      print(f"Error from read_sqlite_table of Gsheet_data_download_split: {e}")
      return pd.DataFrame(columns=['photo_name', 'stave_count'])

  def clear_sqlite_table(self):
    try:
      with closing(sqlite3.connect(self.db_path)) as conn:
        with closing(conn.cursor()) as cursor:
          delete_qry = f"""
          DELETE FROM {self.db_table_name}
          """
          cursor.execute(delete_qry)
          conn.commit()
          print("Succesfully cleared sqlite table")
    except Exception as e:
      print(f"Error from clear_sqlite_table of Gsheet_data_download_split: {e}")
      return False

  def update_table_with_current_img_download(self, photo_name, stave_count):
    try:
      with closing(sqlite3.connect(self.db_path)) as conn:
        with closing(conn.cursor()) as cursor:
          insert_qry = f"""
          INSERT INTO {self.db_table_name} (photo_name, stave_count)
          VALUES ('{photo_name}', {stave_count})
          """
          cursor.execute(insert_qry)
          conn.commit()
          print("Succesfully updated sqlite table")
    except Exception as e:
        print(f"An error occurred while downloading the Google Drive image: {e}")

  ###End sqlite section

In [99]:
#replace these two paths with your own paths on google drive accessed through colab.
SHEET_PATH=userdata.get('gSheetPath')
DOWNLOAD_PATH=""

In [100]:
gsheetObj = Gsheet_data_download(gSheet_path=SHEET_PATH, download_dir_path=DOWNLOAD_PATH, db_path=db_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[]


In [ ]:
# gsheetObj.download_imgs_from_split_dfs()
gsheetObj.download_imgs_without_split()

In [102]:
gsheetObj.read_sqlite_table()

,photo_name,stave_count
0,17289132322424750858174835555844 - Brandon Wagner,609
1,17289134066225452634665236915044 - Brandon Wagner,655
2,17289178627772986676454615492992 - Brandon Wagner,624
3,1728932532448232548761055507990 - Brandon Wagner,645
4,17289325762097634202431665002761 - Brandon Wagner,628
...,...,...
702,17393820395111282713797021226536 - Brandon Wagner,698
703,17394566241271037545571484403264 - Brandon Wagner,715
704,17394566717609020192937876650558 - Brandon Wagner,750
705,17394575149847530031260701566397 - Brandon Wagner,767
